### Notes
- RNN
- One-hot encoding
- Stop character at end of each paper title 

In [77]:
import json
import pandas as pd
import numpy as np
import os

# df = pd.read_json('86.json')
# df

In [60]:
# row = df.loc[0, 'items']

# for key in row.keys():
#     print(key)

indexed
reference-count
publisher
issue
content-domain
short-container-title
published-print
DOI
type
created
page
source
is-referenced-by-count
title
prefix
volume
member
container-title
original-title
link
deposited
score
subtitle
short-title
issued
references-count
URL
relation
ISSN
issn-type


In [61]:
row['title']

['THE ORIGINAL OF MULTIPLE LIPOMAS.']

In [96]:
import re

titles = ""
cwd = os.getcwd()
dir_path = cwd + "/data/"
files = os.listdir(dir_path)
allowed_pattern = r"[^A-Z0-9?().:; ]"

for file in files:
    if file.endswith('.json'):        
        df = pd.read_json(dir_path + file)
    for i in range(len(df)):
        row = df.loc[i, 'items']
        title = row['title']
        title_upper = title[0].upper()
        title_upper = re.sub(allowed_pattern, "", title_upper)
        titles = titles + title_upper + "+"


In [97]:
mean = sum(lens) / len(lens)

print("Max title length: ", max(lens))
print("Min title length: ", min(lens))
print("Average title length: ", int(mean))


Max title length:  295
Min title length:  8
Average title length:  35


In [98]:
chars = sorted(list(set(titles)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [99]:
# Summarize data
n_chars = len(titles)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  676869
Total Vocab:  44


In [100]:
# Prepare dataset of input to output pairs encoded as integers
seq_length = 100
dataX, dataY = [], []
for i in range(0, n_chars - seq_length, 1):
    seq_in = titles[i:i + seq_length]
    seq_out = titles[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  676769


In [101]:
import torch

# Reshape X to be [samples, time steps, features]
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X / float(n_vocab) # works better as number between 0 and 1
y = torch.tensor(dataY)
print(X.shape, y.shape)

torch.Size([676769, 100, 1]) torch.Size([676769])


In [102]:
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

# Define LSTM model
class CharModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=2, batch_first=True, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(256, n_vocab)
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        x = self.linear(self.dropout(x))
        return x

In [ ]:
# Create and run model
n_epochs = 25
batch_size = 128
model = CharModel()

optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(X, y), shuffle=True, batch_size=batch_size)

best_model = None
best_loss = np.inf
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #Validation
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            loss += loss_fn(y_pred, y_batch)
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch: %d: Cross-entropy: %.4f" % (epoch, loss))

Epoch: 0: Cross-entropy: 1482108.3750
Epoch: 1: Cross-entropy: 1283136.3750
Epoch: 2: Cross-entropy: 1133733.5000
Epoch: 3: Cross-entropy: 1067214.7500
Epoch: 4: Cross-entropy: 1020781.3750
Epoch: 5: Cross-entropy: 986601.9375
Epoch: 6: Cross-entropy: 968193.4375


In [73]:
torch.save([best_model, char_to_int], "model_25_12000.pth")